In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

# Loading the dataset

In [2]:
iris = load_iris()

In [3]:
iris.data.shape, iris.target.shape, iris.target_names

((150, 4), (150,), array(['setosa', 'versicolor', 'virginica'],
       dtype='<U10'))

In [4]:
names = iris.target_names

#### - pre-processing

In [5]:
x_train, x_test, y_train, y_test= train_test_split(iris.data, iris.target)

In [6]:
for x in [x_train, y_train, x_test, y_test]:
    print (x.shape)

(112, 4)
(112,)
(38, 4)
(38,)


# Custom Shallow Neural Network

Using the pre-implemented autograde machaine and optimazation algorithm SGD for classification
``` python
torch.autograde
torch.optim.SGD
```

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parameter as parameter
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [8]:
class NN(nn.Module):
    def __init__(self, x_n, y_n):
        """
        
        Arguments:
            x_n (int): length of the input vector
            y_n (int): length of the output vecor
        """
        super(NN, self).__init__()
        # define the input size
        self.x_n = x_n
        # defing the output size
        self.y_n = y_n
        # set constant seed to controle the randomness 
        torch.manual_seed(10)
        # define the weights
        self.w = parameter.Parameter(torch.rand(x_n, y_n, requires_grad=True))
        # define the bias and init it with zeros
        self.b = parameter.Parameter(torch.zeros(1, y_n, requires_grad=True))
        # init the weights
        nn.init.xavier_uniform(self.w)
        # register the parametes in the model
        self.params = nn.ParameterList([self.w, self.b])
        # define lossfunction
        self.criterion = nn.CrossEntropyLoss()
        # define an optimization algorithm 
        self.optimizer = optim.SGD(self.parameters(), lr=0.01)
    
    def forward(self, x):
        """ 
        
        compute the forward pass 
        
        Arguments:
            x (torch.Tensor): the input tensor, shape (m, n_x).
        
        Returns
            torch.Tensor: the output of the network
        """
        return F.softmax(x.mm(self.w) + self.b)
    
    def epoch(self, x, labels):
        """
        perform one epoch of training
        
        Arguments:
            x (torch.Tensor): the input tensor, shape (m, n_x).
            labels (torch.LongTensor): the ground truth, shape (m, 1).
        """
        # init the gradients
        self.optimizer.zero_grad()
        # forward
        pred = self(x)
        # compute the loss 
        loss = self.criterion(pred, labels)
        # backward pass (computing the gradients)
        loss.backward()
        # updating the parmeters according the selected optimization algorithm
        self.optimizer.step()
        
    def train(self, epochs, x, labels):
        """
        perform the training
        
        Arguments:
            epochs (int): number of epochs.
            x (iterator): the input tensor, shape (m, n_x).
            labels (iterator): the ground truth, shape (m, 1).
        """
        # convert the input into `torch.Tensor`
        x = torch.Tensor(x)
        # convert the labels into  `torch.LongTensor`
        labels = torch.LongTensor(labels)
        # perform the the training procedure epochs times.
        for i in range(epochs):
            self.epoch(x, labels)
            
    def evaluate(self, x, labels):
        """ 
        compute the classification accuracy of the network on some dataset 
        
        Arguments:
            x (iterator): the input tensor, shape (m, n_x).
            labels (iterator): the ground truth, shape (m, 1).
        
        Returns
            float: return the accuracy of the model
        """
        x = torch.Tensor(x)
        labels = torch.LongTensor(labels)
        # compute the output fo the network on the given input 
        preds = self(x)
        # return the accuracy of the model
        return (preds.argmax(dim=1) == labels).sum().item()/len(labels)

In [9]:
start = time.time()
nn = NN(4, 3)
nn.train(3000, x_train, y_train)
print("traning time:", time.time() - start)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


traning time: 1.122143030166626


In [10]:
nn.evaluate(x_train, y_train)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.9732142857142857

In [11]:
nn.evaluate(x_test, y_test)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.9473684210526315

# Building Simple GD optimizer

In [24]:
from torch.optim import Optimizer
from torch.optim.optimizer import required
import torch.nn as nn

In [28]:
class GD(Optimizer):
    """
    simple gradient decient optimizer, extends the base optimizer `torch.optim.Optimizer`
    
    Arguments:
        params (iterable): iterable of parameters to optimize
        lr (float): learning rate        
    """
    def __init__(self, params, lr=required):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        defaults = dict(lr=lr)
        super(GD, self).__init__(params, defaults)
        
    def __setstate__(self, state):
        """
        called when the model is beaing loading from file

        TODO: needs testing.
        """
        super(SGD, self).__setstate__(state)

    def step(self):
        """
        takes single Gradient Descent(GD) step.
        """
        # loop over all param groups
        for group in self.param_groups:
            # loop over every parameter in that group
            for p in group['params']:
                # skip if p has no effect on the output
                if p.grad is None:
                    continue
                # update the parameter
                d_p = p.grad.data
                p.data.add_(-group['lr'] * d_p)

In [29]:
class NN_GD(nn.Module):
    def __init__(self, x_n, y_n):
        """
        
        Arguments:
            x_n (int): length of the input vector
            y_n (int): length of the output vecor
        """
        super(NN_GD, self).__init__()
        # define the input size
        self.x_n = x_n
        # defing the output size
        self.y_n = y_n
        # set constant seed to controle the randomness 
        torch.manual_seed(10)
        # define the weights
        self.w = parameter.Parameter(torch.rand(x_n, y_n, requires_grad=True))
        # define the bias and init it with zeros
        self.b = parameter.Parameter(torch.zeros(1, y_n, requires_grad=True))
        # init the weights
        nn.init.xavier_uniform(self.w)
        # register the parametes in the model
        self.params = nn.ParameterList([self.w, self.b])
        # define lossfunction
        self.criterion = nn.CrossEntropyLoss()
        # define an optimization algorithm 
        self.optimizer = GD(self.parameters(), lr=0.01)
    
    def forward(self, x):
        """ 
        
        compute the forward pass 
        
        Arguments:
            x (torch.Tensor): the input tensor, shape (m, n_x).
        
        Returns
            torch.Tensor: the output of the network
        """
        return F.softmax(x.mm(self.w) + self.b)
    
    def epoch(self, x, labels):
        """
        perform one epoch of training
        
        Arguments:
            x (torch.Tensor): the input tensor, shape (m, n_x).
            labels (torch.LongTensor): the ground truth, shape (m, 1).
        """
        # init the gradients
        self.optimizer.zero_grad()
        # forward
        pred = self(x)
        # compute the loss 
        loss = self.criterion(pred, labels)
        # backward pass (computing the gradients)
        loss.backward()
        # updating the parmeters according the selected optimization algorithm
        self.optimizer.step()
        
    def train(self, epochs, x, labels):
        """
        perform the training
        
        Arguments:
            epochs (int): number of epochs.
            x (iterator): the input tensor, shape (m, n_x).
            labels (iterator): the ground truth, shape (m, 1).
        """
        # convert the input into `torch.Tensor`
        x = torch.Tensor(x)
        # convert the labels into  `torch.LongTensor`
        labels = torch.LongTensor(labels)
        # perform the the training procedure epochs times.
        for i in range(epochs):
            self.epoch(x, labels)
            
    def evaluate(self, x, labels):
        """ 
        compute the classification accuracy of the network on some dataset 
        
        Arguments:
            x (iterator): the input tensor, shape (m, n_x).
            labels (iterator): the ground truth, shape (m, 1).
        
        Returns
            float: return the accuracy of the model
        """
        x = torch.Tensor(x)
        labels = torch.LongTensor(labels)
        # compute the output fo the network on the given input 
        preds = self(x)
        # return the accuracy of the model
        return (preds.argmax(dim=1) == labels).sum().item()/len(labels)

In [30]:
start = time.time()
lg = NN_GD(4, 3)
lg.train(3000, x_train, y_train)
print("training time:", time.time() - start)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


training time: 0.7155048847198486


In [31]:
lg.evaluate(x_test, y_test)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.9473684210526315

In [32]:
lg.evaluate(x_train, y_train)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.9732142857142857